In [2]:
from main.compiling.compilers.AncillaPerCheckCompiler import AncillaPerCheckCompiler
from main.compiling.noise.models.CircuitLevelNoise import CircuitLevelNoise
from main.compiling.noise.models.superconducting_inspired_noise import SuperconductingInspired
from main.compiling.syndrome_extraction.extractors.ancilla_per_check.mixed.CxCyCzExtractor import CxCyCzExtractor
from main.codes.tic_tac_toe.HoneycombCode import HoneycombCode
from main.codes.tic_tac_toe.gauge_honeycomb_code import GaugeHoneycombCode
from main.codes.tic_tac_toe.gauge_floquet_colour_code import GaugeFloquetColourCode
from main.codes.tic_tac_toe.FloquetColourCode import FloquetColourCode
from main.codes.tic_tac_toe.TicTacToeCode import TicTacToeCode
from main.utils.enums import State
from main.building_blocks.pauli import Pauli
from main.building_blocks.pauli.PauliLetter import PauliLetter
import stim
import sinter 
import matplotlib.pyplot as plt
import numpy as np
from beliefmatching import BeliefMatchingSinterDecoder
from main.utils.Colour import Blue, Green, Red
import hashlib
import os
from main.building_blocks.detectors.Stabilizer import Stabilizer
from pathlib import Path
from main.building_blocks.logical.LogicalOperator import LogicalOperator



class HoneycombCodeZ(TicTacToeCode):
    """Adaptation of the Honeycomb code where the measurement order has been permuted
    """
    def __init__(self, distance: int):
        tic_tac_toe = [
                (Blue, PauliLetter('Z')),
                (Red, PauliLetter('X')),
                (Green, PauliLetter('Y'))]

        super().__init__(distance, tic_tac_toe)

# load the data.
# seperate running and loading.



# Help functions

In [3]:



def calculate_threshold(code_name, distances, observable_index, layer_factor,pers,noise_model):
    tasks = []
    if code_name == "HoneycombCodeX":
        code = HoneycombCode
        measurement_rounds = 3
    elif code_name == "HoneycombCodeZ":
        code = HoneycombCodeZ
        measurement_rounds = 3
    elif code_name == "GaugeHoneycombCodeX":
        code = GaugeHoneycombCode
        gauge_factor = 2
        measurement_rounds = 6
    elif code_name == "FloquetColourCodeX":
        code = FloquetColourCode
        measurement_rounds = 6
    elif code_name == "GaugeFloquetColourCodeX":
        code = GaugeFloquetColourCode
        gauge_factor = 2
        measurement_rounds = 12


    for d in distances:
        for index, p in enumerate(pers):

            # generate circuit
#            if code_name[-1] == 'X':
            if code_name in ["GaugeHoneycombCodeX", "GaugeFloquetColourCodeX", "GaugeFloquetColourCodeZ", "GaugeHoneycombCodeZ"]:
                circuit = load_or_create_stim_circuit(code(d,gauge_factor), observable_index,layer_factor*d, p, code_name[-1], noise_model)
            else:
                circuit = load_or_create_stim_circuit(code(d), observable_index,layer_factor*d, p, code_name[-1], noise_model)
 #           elif code_name[-1] == 'Z':
  #              circuit = get_Z_stim_circuit(code(d, gauge_factor), observable_index, layer_factor*d, p)
            tasks.append(sinter.Task(
                    circuit=circuit,
                    detector_error_model=circuit.detector_error_model(decompose_errors=True, approximate_disjoint_errors=True, ignore_decomposition_failures=True),
                    json_metadata={
                        'code': code_name,
                        'distance': d,
                        'p': p,
                        'layers': layer_factor*d,
                        'r': layer_factor,
                        'measurement_rounds': measurement_rounds,
                        'total_measurement_rounds': d * layer_factor * measurement_rounds,
                        'noise_model': noise_model}))

                        

    samples = sinter.collect(
        tasks=tasks,
        hint_num_tasks=len(tasks),
        num_workers=10,
        max_shots=10_000,
        max_errors=25,
        decoders=['beliefmatching'],
        custom_decoders={'beliefmatching': BeliefMatchingSinterDecoder()},
        print_progress=True,
        save_resume_filepath=f'./resume_15_2/data.csv',)
    
    # Print samples as CSV data.
    
    return(samples)

def load_or_create_stim_circuit(code, observable_index, layers, p, pauli_type, noise_model):
    hash_fields = (type(code).__name__, code.distance, observable_index, layers, p,pauli_type, noise_model)
    hash_input = str(hash_fields).encode('UTF-8')
    hashed = int(hashlib.md5(hash_input).hexdigest(), 16)
    filepath = '../stim_circuits/'
    filepath = Path(f"../stim_circuits/{hashed}.stim")
    if filepath.is_file():
    #    if code.distance==4:
    #        os.system(f"rm ../stim_circuits/{hashed}.stim")

        stim_circuit = stim.Circuit.from_file(filepath)
    else:
        if pauli_type == 'X':
            stim_circuit = get_X_stim_circuit(code, observable_index, layers, p, noise_model)
        elif pauli_type == 'Z':
            stim_circuit = get_Z_stim_circuit(code, observable_index, layers, p)
        stim_circuit.to_file(filepath)
    return stim_circuit


def get_X_stim_circuit(code,observable_index,layers,p,noise_model):

    if noise_model == "CircuitLevelNoise":
        compiler = AncillaPerCheckCompiler(CircuitLevelNoise(p,p,p,p,p), syndrome_extractor=CxCyCzExtractor())
    elif noise_model == "SuperconductingNoise":
        compiler = AncillaPerCheckCompiler(SuperconductingInspired(p), syndrome_extractor=CxCyCzExtractor())

    data_qubits = code.data_qubits.values()
    data_qubit_initial_states = {qubit: State.Plus for qubit in data_qubits}
    final_measurements = [Pauli(qubit, PauliLetter('X')) for qubit in data_qubits]
    compiler.add_ancilla_qubits(code)
    
    initial_stabilizers = []
    for check in code.check_schedule[0]:
        initial_stabilizers.append(Stabilizer([(0,check)],0))
    
    if observable_index == 3:
        compiler.add_ancilla_qubits(code)
        logical_observables = [LogicalOperator([Pauli(qubit, PauliLetter('Z')) for qubit in code.ancilla_qubits.values()])]
        stim_circuit: stim.Circuit = compiler.compile_to_stim(code,layers=layers, initial_stabilizers=initial_stabilizers,final_measurements=final_measurements,logical_observables=logical_observables, stability=True)
    else:
        logical_observables = [code.logical_qubits[observable_index].x]#, code.logical_qubits[1].x]

        stim_circuit: stim.Circuit = compiler.compile_to_stim(code,layers=layers, initial_stabilizers=initial_stabilizers,final_measurements=final_measurements,logical_observables=logical_observables)

#    stim_circuit: stim.Circuit = compiler.compile_to_stim(code,layers=layers, initial_stabilizers=initial_stabilizers,final_measurements=final_measurements,logical_observables=logical_observables)
    return(stim_circuit)

def get_Z_stim_circuit(code,observable_index,layers,p,noise_model):
    if noise_model == "CircuitLevelNoise":
        compiler = AncillaPerCheckCompiler(CircuitLevelNoise(p,p,p,p,p), syndrome_extractor=CxCyCzExtractor())
    elif noise_model == "SuperconductingNoise":
        compiler = AncillaPerCheckCompiler(SuperconductingInspired(p), syndrome_extractor=CxCyCzExtractor())
    data_qubits = code.data_qubits.values()
    final_measurements = [Pauli(qubit, PauliLetter('Z')) for qubit in data_qubits]
    logical_observables = [code.logical_qubits[observable_index].z]
    initial_stabilizers = []
    for check in code.check_schedule[0]:
        initial_stabilizers.append(Stabilizer([(0,check)],0))
    stim_circuit: stim.Circuit = compiler.compile_to_stim(code,layers=layers, initial_stabilizers=initial_stabilizers, final_measurements=final_measurements,logical_observables=logical_observables)
    return(stim_circuit)

In [3]:
#code = HoneycombCode(4)
#circuit = get_X_stim_circuit(code, 3, 2, 0.1, "CircuitLevelNoise")


testing
heree
6 round
7 round
8 round2
here
[{'product.word': {'word': 'ZZ', 'sign': 1}, 'anchor': (7, 7), 'colour': {'name': 'blue'}, 'paulis': {(1, -1): {'qubit': {'coords': (8, 6)}, 'letter': {'letter': 'Z', 'sign': 1}}, (-1, 1): {'qubit': {'coords': (6, 8)}, 'letter': {'letter': 'Z', 'sign': 1}}}}, {'product.word': {'word': 'ZZ', 'sign': 1}, 'anchor': (1, 7), 'colour': {'name': 'blue'}, 'paulis': {(1, 1): {'qubit': {'coords': (2, 8)}, 'letter': {'letter': 'Z', 'sign': 1}}, (-1, -1): {'qubit': {'coords': (0, 6)}, 'letter': {'letter': 'Z', 'sign': 1}}}}, {'product.word': {'word': 'ZZ', 'sign': 1}, 'anchor': (4, 4), 'colour': {'name': 'blue'}, 'paulis': {(-2, 0): {'qubit': {'coords': (2, 4)}, 'letter': {'letter': 'Z', 'sign': 1}}, (2, 0): {'qubit': {'coords': (6, 4)}, 'letter': {'letter': 'Z', 'sign': 1}}}}, {'product.word': {'word': 'ZZ', 'sign': 1}, 'anchor': (13, 1), 'colour': {'name': 'blue'}, 'paulis': {(1, -1): {'qubit': {'coords': (14, 0)}, 'letter': {'letter': 'Z', 'sign': 1}}

TypeError: list indices must be integers or slices, not Qubit

In [8]:
code.ancilla_qubits

{(10, 2): {'coords': (10, 2)},
 (4, 8): {'coords': (4, 8)},
 (19, 11): {'coords': (19, 11)},
 (7, 3): {'coords': (7, 3)},
 (1, 9): {'coords': (1, 9)},
 (1, 11): {'coords': (1, 11)},
 (22, 10): {'coords': (22, 10)},
 (19, 1): {'coords': (19, 1)},
 (19, 5): {'coords': (19, 5)},
 (7, 9): {'coords': (7, 9)},
 (1, 5): {'coords': (1, 5)},
 (13, 11): {'coords': (13, 11)},
 (13, 7): {'coords': (13, 7)},
 (22, 6): {'coords': (22, 6)},
 (22, 2): {'coords': (22, 2)},
 (1, 3): {'coords': (1, 3)},
 (13, 3): {'coords': (13, 3)},
 (4, 4): {'coords': (4, 4)},
 (19, 3): {'coords': (19, 3)},
 (16, 8): {'coords': (16, 8)},
 (1, 7): {'coords': (1, 7)},
 (13, 9): {'coords': (13, 9)},
 (19, 9): {'coords': (19, 9)},
 (7, 11): {'coords': (7, 11)},
 (1, 1): {'coords': (1, 1)},
 (4, 0): {'coords': (4, 0)},
 (10, 10): {'coords': (10, 10)},
 (13, 1): {'coords': (13, 1)},
 (7, 1): {'coords': (7, 1)},
 (7, 7): {'coords': (7, 7)},
 (16, 0): {'coords': (16, 0)},
 (13, 5): {'coords': (13, 5)},
 (10, 6): {'coords': (10

In [4]:
#gauge_fcc_X1_samples = calculate_threshold('GaugeFloquetColourCodeX',[4,8,12], 1,3, [0.00075, 0.001],'SuperconductingNoise')
gauge_hcc_X1_samples = calculate_threshold('GaugeHoneycombCodeX',[4,8], 1,6, [ 0.0005, 0.00025],'SuperconductingNoise')
fcc_X1_samples = calculate_threshold('FloquetColourCodeX',[4,8], 1,6, [0.00025, 0.0005],'SuperconductingNoise')
hcc_X1_samples = calculate_threshold('HoneycombCodeX',[4,8], 1,12, [0.00025, 0.0005],'SuperconductingNoise')

gauge_fcc_X1_samples = calculate_threshold('GaugeFloquetColourCodeX',[12], 1,3, [0.0005],'SuperconductingNoise')
gauge_hcc_X1_samples = calculate_threshold('GaugeHoneycombCodeX',[12], 1,6, [0.0005],'SuperconductingNoise')
fcc_X1_samples = calculate_threshold('FloquetColourCodeX',[12], 1,6, [0.0005],'SuperconductingNoise')
hcc_X1_samples = calculate_threshold('HoneycombCodeX',[12], 1,12, [0.0005],'SuperconductingNoise')

#gauge_fcc_X1_samples = calculate_threshold('GaugeFloquetColourCodeX',[4,8,12], 1,2, [0.0005, 0.00075, 0.001],'CircuitLevelNoise')
#gauge_hcc_X1_samples = calculate_threshold('GaugeHoneycombCodeX',[4, 8,12], 1,4, [0.0005, 0.00075, 0.001],'CircuitLevelNoise')
#fcc_X1_samples = calculate_threshold('FloquetColourCodeX',[4,8,12], 1,4, [0.0005, 0.00075, 0.001],'CircuitLevelNoise')
#hcc_X1_samples = calculate_threshold('HoneycombCodeX',[4,8,12], 1,8, [0.0005, 0.00075, 0.001],'CircuitLevelNoise')

gauge_fcc_X1_samples = calculate_threshold('GaugeFloquetColourCodeX',[4,8], 1,2, [0.00025],'CircuitLevelNoise')
gauge_hcc_X1_samples = calculate_threshold('GaugeHoneycombCodeX',[4, 8], 1,4, [0.00025],'CircuiteLevelNoise')
fcc_X1_samples = calculate_threshold('FloquetColourCodeX',[4,8], 1,4, [0.00025],'CircuitLevelNoise')
hcc_X1_samples = calculate_threshold('HoneycombCodeX',[4,8], 1,8, [0.00025],'CircuitLevelNoise')


Starting workers...
Finding work...
4 cases left:
    beliefmatching processes=1  ~core_mins_left=None    shots_left=10000    errors_left=25      {code=GaugeHoneycombCodeX,distance=4,layers=24,measurement_rounds=6,noise_model=SuperconductingNoise,p=0.0005,r=6,total_measurement_rounds=144}
    beliefmatching processes=1  ~core_mins_left=None    shots_left=10000    errors_left=25      {code=GaugeHoneycombCodeX,distance=4,layers=24,measurement_rounds=6,noise_model=SuperconductingNoise,p=0.00025,r=6,total_measurement_rounds=144}
    beliefmatching processes=1  ~core_mins_left=None    shots_left=10000    errors_left=25      {code=GaugeHoneycombCodeX,distance=8,layers=48,measurement_rounds=6,noise_model=SuperconductingNoise,p=0.0005,r=6,total_measurement_rounds=288}
    beliefmatching processes=1  ~core_mins_left=None    shots_left=10000    errors_left=25      {code=GaugeHoneycombCodeX,distance=8,layers=48,measurement_rounds=6,noise_model=SuperconductingNoise,p=0.00025,r=6,total_measurement_

UnboundLocalError: local variable 'compiler' referenced before assignment